In [9]:
import json
from collections import defaultdict
from os import listdir
from os.path import isfile, join

test_result = []
prediction_dir = "predictions"
files = [f for f in listdir(prediction_dir) if isfile(join(prediction_dir, f))]

In [25]:
file = files[10]
entities = defaultdict(set)
relations = defaultdict(set)
match_summary = {}
test_id = file.split('/')[-1].split('.')[0]

with open(join(prediction_dir, file), 'r') as f:
    lines = f.readlines()
    
lines

['CLU\tArsenal\n',
 'SCO\t3-0\n',
 'CLU\tArsenal\n',
 'TME\t8\n',
 'PER\tOzil\n',
 'PER\tMustafi\n',
 'CLU\tArsenal\n',
 'CLU\tArsenal\n',
 'TME\t8\n',
 'PER\tAubameyang\n',
 'PER\tMaitland-Niles\n',
 'CLU\tArsenal\n',
 'TME\t59\n',
 'PER\tMkhitaryan\n',
 'CLU\tArsenal\n',
 'TME\t77\n',
 'SCO\t3-0\n',
 'CLU\tArsenal\n',
 'SCO\t3-0\n',
 'SCO\t3-0\n',
 'SCOP\tAubameyang\tArsenal\n',
 'SCOT\t8\tOzil\n',
 'SCOT\t8\tMustafi\n',
 'SCOP\tOzil\tArsenal\n',
 'SCOP\tMustafi\tArsenal\n',
 'SCOP\tOzil\tArsenal\n',
 'SCOP\tMustafi\tArsenal\n',
 'SCOT\t59\tAubameyang\n',
 'SCOP\tMkhitaryan\tArsenal\n',
 'SCOT\t77\tMkhitaryan\n',
 'SCOP\tMkhitaryan\tArsenal\n']

In [22]:
for line in lines:
    line = line.strip()
    tokens = line.split("\t")
    if len(tokens) == 2:
        entities[tokens[0]].add(tokens[1])
    else:
        relations[tokens[0]].add((tokens[1], tokens[2]))
    
print(*entities)
print()
print(*relations)

CLU SCO TME PER

SCOP SCOT


In [16]:
lines

['CLU\tHải_Phòng\n',
 'CLU\tQuảng_Nam\n',
 'CLU\tHải_Phòng\n',
 'PER\tStevens\n',
 'CLU\tHải_Phòng\n',
 'TME\t26\n',
 'PER\tStevens\n',
 'PER\tFagan\n',
 'CLU\tHải_Phòng\n',
 'CLU\tHải_Phòng\n',
 'TME\t26\n',
 'TME\t10 phút sau đó\n',
 'TME\t45 phút hiệp 2\n',
 'SCO\t2-0\n',
 'COMP\tQuảng_Nam\tHải_Phòng\n',
 'DEFE\tQuảng_Nam\tHải_Phòng\n',
 'SCOT\t26\tStevens\n',
 'SCOP\tStevens\tHải_Phòng\n',
 'SCOT\t26\tStevens\n',
 'SCOT\t10 phút sau đó\tFagan\n',
 'SCOP\tStevens\tHải_Phòng\n',
 'SCOP\tFagan\tHải_Phòng\n']

In [19]:
if len(entities.get('CLU', [])) == 2:
    team1, team2 = tuple(entities['CLU'])
team1, team2

('Quảng_Nam', 'Hải_Phòng')

In [ ]:

        

        # s1 > s2
        s1, s2, st1, st2 = 0, 0, 0, 0
        team1, team2 = "", "" 

        for score in entities.get('SCO', []):
            try:
                s = score.strip().split('-')
                st1, st2 = int(s[0]), int(s[1])
            except:
                try:
                    s = score.strip().split('–')
                    st1, st2 = int(s[0]), int(s[1])
                except:
                    pass 

            if st1 + st2 > s1 + s2:
                s1, s2 = max(st1, st2), min(st1, st2)

        # check if there is a winner
        flag = False
        for club1, club2 in relations.get('DEFE', []):
            team1, team2 = club1, club2 
            flag = True 

        # check if it is a tie
        if not flag:
            for club1, club2 in relations.get('DEFE', []):
                team1, team2 = club1, club2
                flag = True 

        # check if we have another clue
        if not flag:
            for club1, club2 in relations.get('COMP', []):
                team1, team2 = club1, club2

        if not flag:
            if len(entities.get('CLU', [])) == 2:
                team1, team2 = tuple(entities['CLU'])

        match_summary["players"] = {
            "team1": team1,
            "team2": team2
        }

        match_summary["score_board"] = {
            "score1": str(s1),
            "score2": str(s2) 
        }

        # get score list 
        score_list = []
        score_player = {}

        for player, club in relations.get('SCOP', []):
            score_player[player] = club 

        for time, player in relations.get('SCOT', []):
            score_info = {
                "player_name": player,
                "team": score_player.get(player, ""),
                "time": time
            }
            score_list.append(score_info)

        match_summary["score_list"] = score_list

        # get card list 
        card_list = []
        card_player = {}

        for player, club in relations.get('CARP', []):
            card_player[player] = club 

        for time, player in relations.get('CART', []):
            card_info = {
                "player_name": player,
                "team": card_player.get(player, ""),
                "time": time
            }
            card_list.append(card_info)

        match_summary["card_list"] = card_list

        # get substitution list
        sub_list = []
        sub_time = {}

        for player_in, time in relations.get('SUBT', []):
            sub_time[player_in] = time

        for player_in, player_out in relations.get('SUBP', []):
            sub_info = {
                "player_in": player_in,
                "player_out": player_out,
                "time": sub_time.get(player_in, '0')
            }
            sub_list.append(sub_info)

        match_summary["substitution_list"] = sub_list
        test_result.append({
            "test_id": test_id, 
            "match_summary": match_summary
        })

    with open(output_file, 'w') as f:
        for article in test_result:
            json.dump(article, f, ensure_ascii=False)
            f.write('\n')

postprocess('predictions')